Last week, I built simple chatbots that interacted with each other without realizing that LLMs are stateless and limited by context windows. Now, with a better understanding of context windows, I’ve updated my code so the two chatbots can interact using the entire conversation history.

This is also Week2 - Day 1 Exercise

In [ ]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

def messages_creation(user_prompt, agent_prompt, system_prompt):
    messages = [{"role": "system", "content": system_prompt}]
    for user, agent in zip(user_prompt, agent_prompt):
        messages.append({"role": "assistant", "content": agent})
        messages.append({"role": "user", "content": user})
    return messages

NumOfConv = 6
llama_system_prompt = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."
phi_system_prompt = "You are a Harsh, Arrogant chatbot. You take time to agree with \
everything the other person says, or find common ground."
llama_message = ["Hello"]
phi_message = ["Hi"]

def chat_with_llama():
    global NumOfConv
    if NumOfConv == 0:
        return
    NumOfConv = NumOfConv - 1
    messages = messages_creation(phi_message, llama_message, llama_system_prompt)
    OLLAMA_BASE_URL = "http://localhost:11434/v1"
    ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
    response = ollama.chat.completions.create(model="llama3.2:1b", messages=messages)
    message = response.choices[0].message.content  
    display(Markdown(f"### Llama:\n{message}\n"))
    phi_message.append(message)
    chat_with_phi()

def chat_with_phi():
    global NumOfConv
    if NumOfConv == 0:
       return
    NumOfConv = NumOfConv - 1
    messages = messages_creation(llama_message, phi_message,phi_system_prompt)
    OLLAMA_BASE_URL = "http://localhost:11434/v1"
    ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
    response = ollama.chat.completions.create(model="phi3", messages=messages)
    message = response.choices[0].message.content  
    display(Markdown(f"### Phi:\n{message}\n"))
    llama_message.append(message)
    chat_with_llama()

chat_with_phi()